# Advent of Code 2025


# Puzzle - part 1

**--- Day 6: Trash Compactor ---**

After helping the Elves in the kitchen,\
you were taking a break and helping them re-enact a movie scene\
when you over-enthusiastically jumped into the garbage chute!

A brief fall later, you find yourself in a garbage smasher.\
Unfortunately, the door's been magnetically sealed.

As you try to find a way out, you are approached by a family of cephalopods!\
They're pretty sure they can get the door open, but it will take some time.\
While you wait, they're curious if you can help the youngest cephalopod with her [math homework](!https://adventofcode.com/2021/day/18).

Cephalopod math doesn't look that different from normal math.\
The math worksheet (your puzzle input) consists of a list of **problems**;\
each problem has a group of numbers that need to be either **added** (`+`) or **multiplied** (`*`) together.

However, the problems are arranged a little strangely;\
they seem to be presented next to each other in a very long horizontal list. For example:
```
123 328  51 64
 45 64  387 23
  6 98  215 314
*   +   *   +
```

Each problem's numbers are arranged vertically;\
at the bottom of the problem is the symbol for the operation that needs to be performed.\
Problems are separated by a full column of only spaces.\
The left/right alignment of numbers within each problem can be ignored.

So, this worksheet contains four problems:
```
123 * 45 * 6 = 33210
328 + 64 + 98 = 490
51 * 387 * 215 = 4243455
64 + 23 + 314 = 401
```

To check their work,cephalopod students are given the **grand total** of\
adding together all of the answers to the individual problems.\
In this worksheet, the grand total is `33210 + 490 + 4243455 + 401 = 4277556`.

Of course, the actual worksheet is **much** wider.\
You'll need to make sure to unroll it completely so that you can read the problems clearly.

Solve the problems on the math worksheet.\
**What is the grand total found by adding together all of the answers to the individual problems?**

## Input

In [268]:
# Load the input file

with open('input - Day 6.txt', 'r') as file:
    file_input = file.read()


print(file_input[:100])

248  1  77 81  57 19 8276 95 33 474 63 536 58 82 167 7813 2986 88 14  9    9 333  14 51 47 866  32 7


## Input Formatting

Let's extract each row first

In [269]:
from pprint import pprint

input_sections = file_input.split('\n')
print(f"This input has {len(input_sections)} sections\n")

input_numbers       = input_sections[:4]
input_operations    = input_sections[4:]

print(f"Input numbers are {len(input_numbers)} sections long")
print(f"Input operations are {len(input_operations)} sections long")


This input has 5 sections

Input numbers are 4 sections long
Input operations are 1 sections long


Okay let's make the numbers into 4 rows of actual ints.\
We will do it in one go.

In [270]:
import re
from pprint import pprint

math_numbers = []

for input_numbers_row in input_numbers:

    input_numbers_str = re.split(r"\s+", input_numbers_row)
    math_numbers_row = list(map(int, input_numbers_str))

    math_numbers.append(math_numbers_row)

print(f"Thera are {len(math_numbers)} rows of math numbers")
print(f"A single row is {len(math_numbers[0])} numbers long\n")

pprint(math_numbers[0][:15])

Thera are 4 rows of math numbers
A single row is 1000 numbers long

[248, 1, 77, 81, 57, 19, 8276, 95, 33, 474, 63, 536, 58, 82, 167]


Now, let's make the math operations into a list.

In [271]:
math_operations = re.split(r"\s+", input_operations[0])

print(f"'{math_operations[-1]}'")

math_operations = math_operations[:-1]

print(f"There are {len(math_operations)} math operations")

''
There are 1000 math operations


## Solution

It seems rather simple, we iterate over the zipped row.\
Then for each number we apply the operation.\
Add it all together and done.

In [272]:
math_results = []
math_number_columns = zip(*math_numbers)

for (n1, n2, n3 ,n4), operation in zip(math_number_columns, math_operations):

    if operation == "+":
        math_results.append(n1 + n2 + n3 + n4)

    elif operation == "*":
        math_results.append(n1 * n2 * n3 * n4)

    else:
        raise ValueError(f"Unknown operation: {operation}")

print(f"The total sum of all results is {sum(math_results)}")
# 4693419406682

The total sum of all results is 4693419406682


# Puzzle - part 2

**--- Part Two ---**

The big cephalopods come back to check on how things are going.\
When they see that your grand total doesn't match the one expected by the worksheet,\
they realize they forgot to explain how to read cephalopod math.

Cephalopod math is written **right-to-left in columns**.\
Each number is given in its own column,\
with the most significant digit at the top and the least significant digit at the bottom.\
(Problems are still separated with a column consisting only of spaces,\
and the symbol at the bottom of the problem is still the operator to use.)

Here's the example worksheet again:
```
123 328  51 64
 45 64  387 23
  6 98  215 314
*   +   *   +
```

Reading the problems right-to-left one column at a time, the problems are now quite different:

The rightmost problem is `4 + 431 + 623 = 1058`\
The second problem from the right is `175 * 581 * 32 = 3253600`\
The third problem from the right is `8 + 248 + 369 = 625`\
Finally, the leftmost problem is `356 * 24 * 1 = 8544`\
Now, the grand total is `1058 + 3253600 + 625 + 8544 = 3263827.`

Solve the problems on the math worksheet again.\
**What is the grand total found by adding together all of the answers to the individual problems?**

## Input Formatting


We have to reformat our input so that the numbers are vertically aligned.\
Then we will ignore the whitespace in our solution.

What we need to find first are the indices of column borders.\
We will use this regex `\s+` to find all the spaces for each row.\
Then we will find, for each set of indices for a particular border position, which is the most common (overlaps the most).\

For example:
```
953 47  28 68 567 61 1735 82 91 967 84 722 61 82 167 7813 2986 88 14  9    9 333
953 47  28 68 567 61 1735 82 91 967 84 722 61 57 371 5625 1184 12 83  84  91 677
265 18 817 65 752 96 4833 86 78 843 87 512 9  88 355  915 2218 37 773 57  79 412
169 96 722  8 585 85 9438 64 99  69 11 42  3  8  252    4 36   64 816 52 399 29
                                                    /\
```

Let's assume the marked border starts at index `0` (for the sake of simplicity).\
For that border for each row we will get the following**range** of border indices `[0]` `[0]` `[0, 1]` `[0, 1, 2]`.

Note that for each row we will have the same number of matches for our regex `\s+`.\
This is because `47  28` and `18 817` at still a single match thanks to the `+`.

Why does this matter, well... we can enumerate over them,\
meaning that each range of indices will have **unique border index** for that match.
As in first match, second match, third match and so on.\
Of course we still know the precise border index, it means we can simply group the ranges.\
We will take advantage of it and use a dictionary.\
Then we can simply add the ranges the that particular border index.

In [273]:
from collections import Counter
from collections import defaultdict

borders_range_of_indices = defaultdict(list)

# Find column border ranges for each row
for input_numbers_row in input_numbers:
    row_borders = []
    
    for border_idx, match in enumerate(re.finditer(r"\s+", input_numbers_row)):
        
        start = match.start()
        end = match.end()
        
        # If single space, just that index; otherwise, range of indices
        if start == end - 1:
            borders_range_of_indices[border_idx].append(start)
            
        else:
            _indices = list(range(start, end))
            borders_range_of_indices[border_idx].append(_indices)

# Find the most common index for each column border
# We need to find the index that overlaps the most across all rows

borders_indices = []

for border_idx in sorted(borders_range_of_indices.keys()):
    values = borders_range_of_indices[border_idx]

    # Flatten: single ints and lists into one list
    all_indices = []
    for v in values:
        if isinstance(v, list):
            all_indices.extend(v)
        else:
            all_indices.append(v)

    # Find most common and append
    most_common = Counter(all_indices).most_common(1)[0][0]
    borders_indices.append(most_common)

print(f"Found {len(borders_indices)} border indices")

Found 999 border indices


In [274]:
# Split each row according to border indices

math_numbers_str = []

for row in input_numbers:
    columns = []
    prev_end = 0

    for border in borders_indices:
        columns.append(row[prev_end:border])
        prev_end = border + 1

    # Last column (after the final border)
    columns.append(row[prev_end:])

    math_numbers_str.append(columns)

print(f"Number of rows: {len(math_numbers_str)}")
print(f"Number of columns per row: {len(math_numbers_str[0])}")

Number of rows: 4
Number of columns per row: 1000


In [275]:
print(math_numbers_str[0][:20])
print(math_numbers_str[1][:20])
print(math_numbers_str[2][:20])
print(math_numbers_str[3][:20])

['248', ' 1', ' 77', '81', ' 57', '19', '8276', '95', '33', '474', '63', '536', '58', '82', '167', '7813', '2986', '88', '14 ', '9 ']
['953', '47', ' 28', '68', '567', '61', '1735', '82', '91', '967', '84', '722', '61', '57', '371', '5625', '1184', '12', '83 ', '84']
['265', '18', '817', '65', '752', '96', '4833', '86', '78', '843', '87', '512', '9 ', '88', '355', ' 915', '2218', '37', '773', '57']
['169', '96', '722', ' 8', '585', '85', '9438', '64', '99', ' 69', '11', '42 ', '3 ', '8 ', '252', '   4', '36  ', '64', '816', '52']


## Solution

It's easy we just re-use the loop we had.\
Here is what we need to do.

Pre-initialize the array for cephalopod numbers,\
since we can get as little as one or as many as there are digits in a number.

And that's really all there is to it.
Note we don't care about direction of iterations because we have the `" "` that align everything.

In [276]:
from functools import reduce
import operator

math_results = []
math_number_columns_str = zip(*math_numbers_str)

for (s1, s2, s3 ,s4), operation in zip(math_number_columns_str, math_operations):

    # The number of the cephalopod numbers we will get
    max_decimal_places = len(max([s1, s2, s3 ,s4], key=len))

    # Initialize
    cephalopod_numbers = ["" for _ in range(max_decimal_places)]

    for number in [s1, s2, s3, s4]:

        for decimal_place, digit in enumerate(number):

            # Ignore whitespace
            if digit == " ":
                continue

            cephalopod_numbers[decimal_place] += digit

    # Conver strings into integers for math operations
    cephalopod_numbers = [int(cn) for cn in cephalopod_numbers]

    # Math operations
    if operation == "+":
        math_results.append(sum(cephalopod_numbers))

    elif operation == "*":
        math_results.append(reduce(operator.mul, cephalopod_numbers, 1))

    else:
        raise ValueError(f"Unknown operation: {operation}")


print(f"The total sum of all results is {sum(math_results)}")
# 9029931401920

The total sum of all results is 9029931401920
